In [0]:
import numpy as np 
import re
# data processing
import pandas as pd 
# data visualization
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib import style
# Algorithms
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_predict

In [0]:
#Importing the Libraries
# linear algebra
import numpy as np 

# data processing
import pandas as pd 

# data visualization
import seaborn as sns
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import style

# Algorithms
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB


In [0]:
#Getting the Data

test_df = pd.read_csv("/content/test.csv")
train_df = pd.read_csv("/content/train.csv")

In [0]:
#Data Exploration/Analysis
'''

survival:    Survival 
PassengerId: Unique Id of a passenger. 
pclass:    Ticket class     
sex:    Sex     
Age:    Age in years     
sibsp:    # of siblings / spouses aboard the Titanic     
parch:    # of parents / children aboard the Titanic     
ticket:    Ticket number     
fare:    Passenger fare     
cabin:    Cabin number     
embarked:    Port of Embarkation
'''

train_df.info()
train_df.describe()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


Above we can see that 38% out of the training-set survived the Titanic. We can also see that the passenger ages range from 0.4 to 80. On top of that we can already detect some features, that contain missing values, like the ‘Age’ feature.
We need to convert a lot of features into numeric ones later on, so that the machine learning algorithms can process them. Furthermore, we can see that the features have widely different ranges, that we will need to convert into roughly the same scale. We can also spot some more features, that contain missing values (NaN = not a number), that wee need to deal with.

In [0]:
total = train_df.isnull().sum().sort_values(ascending=False)
percent_1 = train_df.isnull().sum()/train_df.isnull().count()*100
percent_2 = (round(percent_1, 1)).sort_values(ascending=False)
missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%'])
missing_data.head(5)

,Total,%
Cabin,687,77.1
Age,177,19.9
Embarked,2,0.2
Fare,0,0.0
Ticket,0,0.0


The Embarked feature has only 2 missing values, which can easily be filled. It will be much more tricky, to deal with the ‘Age’ feature, which has 177 missing values. The ‘Cabin’ feature needs further investigation, but it looks like that we might want to drop it from the dataset, since 77 % of it are missing.

First, I will drop ‘PassengerId’ from the train set, because it does not contribute to a persons survival probability. I will not drop it from the test set, since it is required there for the submission.

In [0]:
train_df = train_df.drop(['PassengerId'], axis=1)



Missing Data:
Cabin:
As a reminder, we have to deal with Cabin (687), Embarked (2) and Age (177). First I thought, we have to delete the ‘Cabin’ variable but then I found something interesting. A cabin number looks like ‘C123’ and the letter refers to the deck. Therefore we’re going to extract these and create a new feature, that contains a persons deck. Afterwords we will convert the feature into a numeric variable. The missing values will be converted to zero. 



In [0]:
import re
deck = {"A": 1, "B": 2, "C": 3, "D": 4, "E": 5, "F": 6, "G": 7, "U": 8}
data = [train_df, test_df]

for dataset in data:
    dataset['Cabin'] = dataset['Cabin'].fillna("U0")
    dataset['Deck'] = dataset['Cabin'].map(lambda x: re.compile("([a-zA-Z]+)").search(x).group())
    dataset['Deck'] = dataset['Deck'].map(deck)
    dataset['Deck'] = dataset['Deck'].fillna(0)
    dataset['Deck'] = dataset['Deck'].astype(int)
# we can now drop the cabin feature
train_df = train_df.drop(['Cabin'], axis=1)
test_df = test_df.drop(['Cabin'], axis=1)


Age:
Now we can tackle the issue with the age features missing values. I will create an array that contains random numbers, which are computed based on the mean age value in regards to the standard deviation and is_null.


In [0]:
data = [train_df, test_df]

for dataset in data:
    mean = train_df["Age"].mean()
    std = test_df["Age"].std()
    is_null = dataset["Age"].isnull().sum()
    # compute random numbers between the mean, std and is_null
    rand_age = np.random.randint(mean - std, mean + std, size = is_null)
    # fill NaN values in Age column with random values generated
    age_slice = dataset["Age"].copy()
    age_slice[np.isnan(age_slice)] = rand_age
    dataset["Age"] = age_slice
    dataset["Age"] = train_df["Age"].astype(int)
train_df["Age"].isnull().sum()

0

Since the Embarked feature has only 2 missing values, we will just fill these with the most common one.

In [0]:
train_df['Embarked'].describe()


count     889
unique      3
top         S
freq      644
Name: Embarked, dtype: object

In [0]:
common_value = 'S'
data = [train_df, test_df]

for dataset in data:
    dataset['Embarked'] = dataset['Embarked'].fillna(common_value)

Converting “Fare” from float to int64, using the “astype()” function pandas provides:

In [0]:
data = [train_df, test_df]

for dataset in data:
    dataset['Fare'] = dataset['Fare'].fillna(0)
    dataset['Fare'] = dataset['Fare'].astype(int)

We will use the Name feature to extract the Titles from the Name, so that we can build a new feature out of that.

In [0]:
data = [train_df, test_df]
titles = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}

for dataset in data:
    # extract titles
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
    # replace titles with a more common title or as Rare
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr',\
                                            'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    # convert titles into numbers
    dataset['Title'] = dataset['Title'].map(titles)
    # filling NaN with 0, to get safe
    dataset['Title'] = dataset['Title'].fillna(0)
train_df = train_df.drop(['Name'], axis=1)
test_df = test_df.drop(['Name'], axis=1)

Convert ‘Sex’ feature into numeric.
genders = {"male": 0, "female": 1}

In [0]:
genders = {"male": 0, "female": 1}
data = [train_df, test_df]

for dataset in data:
    dataset['Sex'] = dataset['Sex'].map(genders)

In [0]:
train_df['Ticket'].describe()


count        891
unique       681
top       347082
freq           7
Name: Ticket, dtype: object

Since the Ticket attribute has 681 unique tickets, it will be a bit tricky to convert them into useful categories. So we will drop it from the dataset.

In [0]:
train_df = train_df.drop(['Ticket'], axis=1)
test_df = test_df.drop(['Ticket'], axis=1)

Convert ‘Embarked’ feature into numeric

In [0]:
ports = {"S": 0, "C": 1, "Q": 2}
data = [train_df, test_df]

for dataset in data:
    dataset['Embarked'] = dataset['Embarked'].map(ports)

Creating Categories:
We will now create categories within the following features:
Age:
Now we need to convert the ‘age’ feature. First we will convert it from float into integer. Then we will create the new ‘AgeGroup” variable, by categorizing every age into a group. Note that it is important to place attention on how you form these groups, since you don’t want for example that 80% of your data falls into group 1.

In [0]:
data = [train_df, test_df]
for dataset in data:
    dataset['Age'] = dataset['Age'].astype(int)
    dataset.loc[ dataset['Age'] <= 11, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 11) & (dataset['Age'] <= 18), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 18) & (dataset['Age'] <= 22), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 22) & (dataset['Age'] <= 27), 'Age'] = 3
    dataset.loc[(dataset['Age'] > 27) & (dataset['Age'] <= 33), 'Age'] = 4
    dataset.loc[(dataset['Age'] > 33) & (dataset['Age'] <= 40), 'Age'] = 5
    dataset.loc[(dataset['Age'] > 40) & (dataset['Age'] <= 66), 'Age'] = 6
    dataset.loc[ dataset['Age'] > 66, 'Age'] = 6

# let's see how it's distributed 
train_df['Age'].value_counts()

0    891
Name: Age, dtype: int64

For the ‘Fare’ feature, we need to do the same as with the ‘Age’ feature. But it isn’t that easy, because if we cut the range of the fare values into a few equally big categories, 80% of the values would fall into the first category. Fortunately, we can use sklearn “qcut()” function, that we can use to see, how we can form the categories.

In [0]:

data = [train_df, test_df]

for dataset in data:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[(dataset['Fare'] > 31) & (dataset['Fare'] <= 99), 'Fare']   = 3
    dataset.loc[(dataset['Fare'] > 99) & (dataset['Fare'] <= 250), 'Fare']   = 4
    dataset.loc[ dataset['Fare'] > 250, 'Fare'] = 5
    dataset['Fare'] = dataset['Fare'].astype(int)
train_df = train_df.drop(['Embarked'], axis=1)
test_df = test_df.drop(['Embarked'], axis=1)

In [0]:

#Split the data into independent 'X' and dependent 'Y' variables
X = np.array(train_df.drop(['Survived'], 1).astype(float))
y = np.array(train_df['Survived'])
kmeans = KMeans(n_clusters=2) # You want cluster the passenger records into 2: Survived or Not survived
kmeans.fit(X)
KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)
correct = 0
for i in range(len(X)):
    predict_me = np.array(X[i].astype(float))
    predict_me = predict_me.reshape(-1, len(predict_me))
    prediction = kmeans.predict(predict_me)
    if prediction[0] == y[i]:
        correct += 1

print(correct/len(X))


0.6891133557800224


In [0]:
########################### 1Support Vector Machine  ####################### 
X_train = train_df.drop("Survived", axis=1)
Y_train = train_df["Survived"]
X_test  = test_df.drop("PassengerId", axis=1).copy()
svm_clf1 = SVC(kernel='linear', C=1)
svm_clf1.fit(X_train, Y_train)
Y_pred = svm_clf1.predict(X_test)

acc_linear_svc1 = round(svm_clf1.score(X_train, Y_train) * 100, 2)
Y_pred = cross_val_predict(svm_clf1, X_train, Y_train, cv=3)
precision_svc1= round(precision_score(Y_train, Y_pred) * 100, 2)
recall_svc1 = round(recall_score(Y_train, Y_pred) * 100, 2)
f1_svc1 = round(f1_score(Y_train, Y_pred) * 100, 2)

#####################2
svm_clf2 = SVC(kernel='linear', C=10)
svm_clf2.fit(X_train, Y_train)
Y_pred = svm_clf2.predict(X_test)
acc_linear_svc2 = round(svm_clf2.score(X_train, Y_train) * 100, 2)

Y_pred = cross_val_predict(svm_clf2, X_train, Y_train, cv=3)
precision_svc2= round(precision_score(Y_train, Y_pred) * 100, 2)
recall_svc2 = round(recall_score(Y_train, Y_pred) * 100, 2)
f1_svc2 = round(f1_score(Y_train, Y_pred) * 100, 2)
########################3
poly_kernel_svm_clf = SVC(kernel="poly", degree=3, coef0=1, C=5)
 
poly_kernel_svm_clf.fit(X_train, Y_train)
acc_linear_svc3 = round(poly_kernel_svm_clf.score(X_train, Y_train) * 100, 2)
Y_pred = cross_val_predict(poly_kernel_svm_clf, X_train, Y_train, cv=3)
precision_poly= round(precision_score(Y_train, Y_pred) * 100, 2)
recall_poly = round(recall_score(Y_train, Y_pred) * 100, 2)
f1_poly = round(f1_score(Y_train, Y_pred) * 100, 2)
##########################4
rbf_kernel_svm_clf = SVC(kernel="rbf", gamma=0.1, C=1000)
rbf_kernel_svm_clf.fit(X_train, Y_train)

acc_linear_svc4 = round(rbf_kernel_svm_clf.score(X_train, Y_train) * 100, 2)
Y_pred = cross_val_predict(rbf_kernel_svm_clf , X_train, Y_train, cv=3)
precision_rbf=round(precision_score(Y_train,Y_pred)* 100, 2)
recall_rbf=round(recall_score(Y_train,Y_pred)* 100, 2)
f1_rbf = round(f1_score(Y_train, Y_pred) * 100, 2)
##########################5 logistic regression ###################
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)

acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
Y_pred = cross_val_predict(logreg , X_train, Y_train, cv=3)
precision_lg=round(precision_score(Y_train,Y_pred)* 100, 2)
recall_lg=round(recall_score(Y_train,Y_pred)* 100, 2)
f1_lg = round(f1_score(Y_train, Y_pred) * 100, 2)
###########################6 naive Bayes ######################## 
gaussian = GaussianNB()
gaussian.fit(X_train, Y_train) 

acc_gaussian = round(gaussian.score(X_train, Y_train) * 100, 2)
Y_pred = cross_val_predict(gaussian , X_train, Y_train, cv=3)
precision_NB=round(precision_score(Y_train,Y_pred)* 100, 2)
recall_NB=round(recall_score(Y_train,Y_pred)* 100, 2)
f1_NB = round(f1_score(Y_train, Y_pred) * 100, 2)
####################### 7decision tree ###################
decision_tree1 = DecisionTreeClassifier() 
decision_tree1.fit(X_train, Y_train)

acc_decision_tree1 = round(decision_tree1.score(X_train, Y_train) * 100, 2)
Y_pred = cross_val_predict(decision_tree1 , X_train, Y_train, cv=3)
precision_DT=round(precision_score(Y_train,Y_pred)* 100, 2)
recall_DT=round(recall_score(Y_train,Y_pred)* 100, 2)
f1_DT = round(f1_score(Y_train, Y_pred) * 100, 2)
##########################8
decision_tree2 = DecisionTreeClassifier(max_depth=3, min_samples_split=0.1, min_samples_leaf=0.1) 
decision_tree2.fit(X_train, Y_train)
   
acc_decision_tree2 = round(decision_tree2.score(X_train, Y_train) * 100, 2)
Y_pred = cross_val_predict(decision_tree2 , X_train, Y_train, cv=3)
precision_DT2=round(precision_score(Y_train,Y_pred)* 100, 2)
recall_DT2=round(recall_score(Y_train,Y_pred)* 100, 2)
f1_DT2 = round(f1_score(Y_train, Y_pred) * 100, 2)
###########################9
decision_tree3 = DecisionTreeClassifier(max_depth=10, min_samples_split=3,min_samples_leaf=5) 
decision_tree3.fit(X_train, Y_train)

acc_decision_tree3 = round(decision_tree3.score(X_train, Y_train) * 100, 2)
Y_pred = cross_val_predict(decision_tree3 , X_train, Y_train, cv=3)
precision_DT3=round(precision_score(Y_train,Y_pred)* 100, 2)
recall_DT3=round(recall_score(Y_train,Y_pred)* 100, 2)
f1_DT3 = round(f1_score(Y_train, Y_pred) * 100, 2)
############################ 10 Random forest #######################
random_forest = RandomForestClassifier(criterion = "gini", 
                                       max_depth= 6,
                                       n_estimators=100, 
                                       max_features='log2', 
                                       )

random_forest.fit(X_train, Y_train)

random_forest_score=(round(random_forest.score(X_train, Y_train)* 100,2))
Y_pred = cross_val_predict(random_forest , X_train, Y_train, cv=3)
precision_rf=round(precision_score(Y_train,Y_pred)* 100, 2)
recall_rf=round(recall_score(Y_train,Y_pred)* 100, 2)
f1_rf = round(f1_score(Y_train, Y_pred) * 100, 2)
################### 11Gradiant boosting with grid search cv #####################


gradientBoosting = GradientBoostingClassifier(min_samples_leaf= 40, n_estimators= 20, min_samples_split= 200, learning_rate= 1, max_depth= 7) 
                                       

gradientBoosting.fit(X_train, Y_train)

GB_score=(round(gradientBoosting.score(X_train, Y_train)* 100,2))
Y_pred = cross_val_predict(gradientBoosting , X_train, Y_train, cv=3)
precision_gb=round(precision_score(Y_train,Y_pred)* 100, 2)
recall_gb=round(recall_score(Y_train,Y_pred)* 100, 2)
f1_gb = round(f1_score(Y_train, Y_pred) * 100, 2)
##################12 Bagging classifier Decision tree  ##################
bag_clf_DT=BaggingClassifier(DecisionTreeClassifier(),n_estimators=100,max_samples=100,bootstrap=True,n_jobs=-1)
bag_clf_DT.fit(X_train, Y_train)

bag_clf_Score_DT=(round(bag_clf_DT.score(X_train, Y_train)* 100,2))
Y_pred = cross_val_predict(bag_clf_DT , X_train, Y_train, cv=3)
precision_bagg_dt=round(precision_score(Y_train,Y_pred)* 100, 2)
recall_bagg_dt=round(recall_score(Y_train,Y_pred)* 100, 2)
f1_bagg_dt = round(f1_score(Y_train, Y_pred) * 100, 2)
##################### 13Bagging naive bayes ###################
bag_clf_NB=BaggingClassifier(GaussianNB() ,n_estimators=100,max_samples=100,bootstrap=True,n_jobs=-1)
bag_clf_NB.fit(X_train, Y_train)

bag_clf_Score_NB=(round(bag_clf_NB.score(X_train, Y_train)* 100,2))
Y_pred = cross_val_predict(bag_clf_NB , X_train, Y_train, cv=3)
precision_bagg_nb=round(precision_score(Y_train,Y_pred)* 100, 2)
recall_bagg_nb=round(recall_score(Y_train,Y_pred)* 100, 2)
f1_bagg_nb = round(f1_score(Y_train, Y_pred) * 100, 2)
####################14 Ada boosting #########################
Ada_boost = AdaBoostClassifier( n_estimators= 400, learning_rate= 0.05, algorithm= 'SAMME.R')                         
Ada_boost.fit(X_train, Y_train)

Ada_score=(round(Ada_boost.score(X_train, Y_train)* 100,2))
Y_pred = cross_val_predict(Ada_boost , X_train, Y_train, cv=3)
precision_ada_boost=round(precision_score(Y_train,Y_pred)* 100, 2)
recall_ada_boost=round(recall_score(Y_train,Y_pred)* 100, 2)
f1_ada_boost = round(f1_score(Y_train, Y_pred) * 100, 2)
##################################15 Voting ##########################
voting_clf = VotingClassifier(
 estimators=[('lr', logreg), ('rf', random_forest), ('svc', svm_clf2), ('DT',decision_tree1),('NB',gaussian)],
 voting='hard'
 )
voting_clf.fit(X_train, Y_train)

voting_score=(round(voting_clf.score(X_train, Y_train)* 100,2))
Y_pred = cross_val_predict(voting_clf , X_train, Y_train, cv=3)
precision_voting=round(precision_score(Y_train,Y_pred)* 100, 2)
recall_voting=round(recall_score(Y_train,Y_pred)* 100, 2)
f1_voting = round(f1_score(Y_train, Y_pred) * 100, 2)
########################################## Result #################
results = pd.DataFrame({
    'Model': ['SVM  linear c=1','svm linear c=10 ','svm poly kernel', 'svm rbf kernel', 'Logistic Regression', 
               'Naive Bayes', 
              'Decision Tree1','Decision Tree2','Decision tree3','random_forest_score','GB_score','bag_clf_Score_DT','bag_clf_Score_NB','Ada_boost','Voting'],
    'Score': [acc_linear_svc1, acc_linear_svc2,acc_linear_svc3, acc_linear_svc4, acc_log, 
             acc_gaussian, 
              acc_decision_tree1,acc_decision_tree2,acc_decision_tree3,random_forest_score,GB_score,bag_clf_Score_DT,bag_clf_Score_NB,Ada_score,voting_score],
              'Precision_Score':[precision_svc1, precision_svc2,precision_poly,precision_rbf,precision_lg,precision_NB,precision_DT,precision_DT2,precision_DT3,precision_rf,precision_gb,precision_bagg_dt,precision_bagg_nb,precision_ada_boost,precision_voting], 
              'Recall_Score':[recall_svc1, recall_svc2,recall_poly,recall_rbf,recall_lg,recall_NB,recall_DT,recall_DT2,recall_DT3,recall_rf,recall_gb,recall_bagg_dt,recall_bagg_nb,recall_ada_boost,recall_voting],
              'F1_Score':[f1_svc1, f1_svc2,f1_poly,f1_rbf,f1_lg,f1_NB,f1_DT,f1_DT2,f1_DT3,f1_rf,f1_gb,f1_bagg_dt,f1_bagg_nb,f1_ada_boost,f1_voting]
               }
              )
              
              
print(results)       
'''results.plot(x= 'Model', kind='barh')
plt.title('Models Performance')
plt.show()'''



/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default 

                  Model  Score  Precision_Score  Recall_Score  F1_Score
0       SVM  linear c=1  78.45            74.84         69.59     72.12
1      svm linear c=10   78.45            74.84         69.59     72.12
2       svm poly kernel  87.88            79.29         71.64     75.27
3        svm rbf kernel  92.70            73.46         66.37     69.74
4   Logistic Regression  81.03            74.56         73.68     74.12
5           Naive Bayes  78.23            67.92         79.24     73.14
6        Decision Tree1  93.04            76.59         66.96     71.45
7        Decision Tree2  78.90            77.10         59.06     66.89
8        Decision tree3  86.53            78.26         68.42     73.01
9   random_forest_score  86.98            81.33         71.35     76.01
10             GB_score  87.88            82.99         71.35     76.73
11     bag_clf_Score_DT  86.42            82.99         69.88     75.87
12     bag_clf_Score_NB  78.34            66.75         76.32   

"results.plot(x= 'Model', kind='barh')\nplt.title('Models Performance')\nplt.show()"